# SQL Practice Problems using sqlite + Pandas + Jupyter notebooks and Northwinds Database 

###### DB: https://docs.microsoft.com/en-us/dotnet/framework/data/adonet/sql/linq/downloading-sample-databases



In [1]:
import sqlite3

# Create a SQL connection to our SQLite database
con = sqlite3.connect("Northwind_large.sqlite")

# check available tables 
import pandas as pd

cur = con.cursor()
cur.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
available_table=(cur.fetchall())


In [2]:
# Problem 32: High value customers: at least 1 order > 10000 in 2015

Q = '''Select
Customers.Id,
Customers.CompanyName,Orders.ID, 
sum(Quantity * UnitPrice) as TAM
From Customer Customers 
    Join "Order" Orders on Orders.CustomerID = Customers.ID 
    Join OrderDetail OrderDetails on Orders.ID= OrderDetails.OrderId 
    Where 
        OrderDate >= "20150101" and OrderDate < "20160101"
        
        Group By 
            Customers.ID,
            Customers.CompanyName,
            Orders.ID
            having TAM > 10000'''

pd.read_sql( Q , con)


,Id,CompanyName,Id,TAM
0,ALFKI,Alfreds Futterkiste,11565,37342.47
1,ALFKI,Alfreds Futterkiste,14496,41685.87
2,ALFKI,Alfreds Futterkiste,14606,21295.56
3,ALFKI,Alfreds Futterkiste,17114,37997.07
4,ALFKI,Alfreds Futterkiste,17765,26304.25
...,...,...,...,...
460,WOLZA,Wolski Zajazd,11670,35814.41
461,WOLZA,Wolski Zajazd,14449,34936.00
462,WOLZA,Wolski Zajazd,15391,42307.49
463,WOLZA,Wolski Zajazd,21074,36276.00


In [3]:
# Problem 33 Customers Ordering total > $15000 or more in 2015


Q = '''Select
Customers.Id,
Customers.CompanyName,
sum(Quantity *UnitPrice* (1-Discount)) as Amt_With_Discount
From Customer Customers 
    Join "Order" Orders on Orders.CustomerID = Customers.ID 
    Join OrderDetail OrderDetails on Orders.ID= OrderDetails.OrderId 
    Where 
        OrderDate >= "20150101" and OrderDate < "20160101"
        
        Group By 
            Customers.ID,
            Customers.CompanyName
            having Amt_With_Discount > 15000 order by 3'''

pd.read_sql( Q , con)

,Id,CompanyName,Amt_With_Discount
0,HUNGC,Hungry Coyote Import Store,28482.80
1,FOLIG,Folies gourmandes,46075.57
2,OLDWO,Old World Delicatessen,52038.56
3,SAVEA,Save-a-lot Markets,53756.33
4,RICSU,Richter Supermarkt,57696.80
...,...,...,...
86,ANTON,Antonio Moreno Taquería,300125.75
87,KOENE,Königlich Essen,318929.59
88,SUPRD,Suprêmes délices,375843.90
89,GOURL,Gourmet Lanchonetes,422673.23


In [4]:
# Problem 34 Orders placed Only on last day of month - we break the problem into two parts
# get last day of month, and show orders where date = last date of month

Q = "SELECT EmployeeId,Id, OrderDate from 'Order' where OrderDate = date(OrderDate,'start of month','+1 month','-1 day') ;"
pd.read_sql( Q , con)

,EmployeeId,Id,OrderDate
0,5,10269,2012-07-31
1,6,10317,2012-09-30
2,4,10343,2012-10-31
3,8,10399,2012-12-31
4,3,10432,2013-01-31
5,8,10460,2013-02-28
6,1,10461,2013-02-28
7,7,10490,2013-03-31
8,8,10491,2013-03-31
9,4,10522,2013-04-30


In [5]:
# Problem 35 for each employee, Orders placed Only on last day of month - we break the problem into two parts
# get last day of month, and show orders where date = last date of month

Q = "SELECT EmployeeId, count(*) from 'Order' where OrderDate = date(OrderDate,'start of month','+1 month','-1 day') group by 1;"
pd.read_sql( Q , con)

,EmployeeId,count(*)
0,1,2
1,2,4
2,3,4
3,4,8
4,5,1
5,6,1
6,7,1
7,8,4
8,9,1


In [6]:
# problem 36 Order ids with most items

Q = "SELECT OrderId, count(*) from OrderDetail group by 1 order by 2 desc limit 10;"
pd.read_sql( Q , con)

,OrderId,count(*)
0,27062,77
1,26911,77
2,26752,77
3,26569,77
4,26465,77
5,26444,77
6,26443,77
7,26433,77
8,26358,77
9,26324,77


In [7]:
# Excercise 38: Double entry item > 60

Q = "SELECT OrderId, Quantity, count(*) as counts from OrderDetail where Quantity >= 60 group by 1,2 having counts = 2;"
pd.read_sql( Q , con)

,OrderId,Quantity,counts
0,10263,60,2
1,10658,70,2
2,10990,65,2
3,11030,100,2


In [11]:
# Details of above Order

Q = '''
WITH proborder as (
select OrderId, Quantity, count(*) as counts from OrderDetail where Quantity >= 60 group by 1,2 having counts = 2
)

SELECT * from OrderDetail where OrderID in (
        select OrderID from proborder);'''
pd.read_sql( Q , con)

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount
0,10263/16,10263,16,13.90,60,0.25
1,10263/24,10263,24,3.60,28,0.00
2,10263/30,10263,30,20.70,60,0.25
3,10263/74,10263,74,8.00,36,0.25
4,10658/21,10658,21,10.00,60,0.00
5,10658/40,10658,40,18.40,70,0.05
6,10658/60,10658,60,34.00,55,0.05
7,10658/77,10658,77,13.00,70,0.05
8,10990/21,10990,21,10.00,65,0.00
9,10990/34,10990,34,14.00,60,0.15


In [14]:
#Excercise 41 Late Orders

Q = '''select * from "Order" where ShippedDate > RequiredDate;'''
pd.read_sql( Q , con)

,Id,CustomerId,EmployeeId,OrderDate,RequiredDate,ShippedDate,ShipVia,Freight,ShipName,ShipAddress,ShipCity,ShipRegion,ShipPostalCode,ShipCountry
0,10264,FOLKO,6,2012-07-24,2012-08-21,2012-08-23,3,25.00,Folk och fä HB,Åkergatan 24,Bräcke,Northern Europe,S-844 67,Sweden
1,10271,SPLIR,6,2012-08-01,2012-08-29,2012-08-30,2,16.00,Split Rail Beer & Ale,P.O. Box 555,Lander,North America,82520,USA
2,10280,BERGS,2,2012-08-14,2012-09-11,2012-09-12,1,25.50,Berglunds snabbköp,Berguvsvägen 8,Luleå,Northern Europe,S-958 22,Sweden
3,10302,SUPRD,4,2012-09-10,2012-10-08,2012-10-09,2,30.00,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,Western Europe,B-6000,Belgium
4,10309,HUNGO,3,2012-09-19,2012-10-17,2012-10-23,1,28.75,Hungry Owl All-Night Grocers,8 Johnstown Road,Cork,British Isles,None,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,27040,SPLIR,2,2015-02-19 21:04:45,2015-03-15 19:52:16,2015-03-21 13:54:13,3,92.50,Split Rail Beer & Ale,P.O. Box 555,Lander,North America,82520,USA
3949,27042,OCEAN,2,2014-06-18 08:26:12,2014-06-18 19:28:10,2014-06-18 23:23:37,2,52.50,Trail's Head Gourmet Provisioners,722 DaVinci Blvd.,Kirkland,North America,98034,USA
3950,27055,SPLIR,9,2015-10-06 15:31:42,2015-10-13 16:23:06,2015-10-16 14:46:08,1,246.00,Toms Spezialitäten,Luisenstr. 48,Münster,Western Europe,44087,Germany
3951,27056,FAMIA,7,2014-07-29 01:31:55,2014-08-01 18:55:28,2014-08-03 20:07:44,3,280.75,Ottilies Käseladen,Mehrheimerstr. 369,Köln,Western Europe,50739,Germany


In [18]:
#Excercise 42 Employee details of late orders


Q = '''
WITH late_orders as 
    (select * from "Order" where ShippedDate > RequiredDate)
    
    select EmployeeId, LastName, count(*) from late_orders a
    inner join Employee b on
    a.EmployeeId = b.Id
    
    group by 1,2
    order by 3
    
    
    
    ;'''
pd.read_sql( Q , con)

,EmployeeId,LastName,count(*)
0,3,Leverling,419
1,8,Callahan,421
2,2,Fuller,432
3,5,Buchanan,432
4,6,Suyama,437
5,7,King,438
6,1,Davolio,446
7,9,Dodsworth,451
8,4,Peacock,477


In [21]:
# Excercise 43,44,45 all orders vs total orders
Q = '''
WITH all_orders as 
    (select *, 
    CASE
        when ShippedDate > RequiredDate then 1 else 0 END as lateorder
    
    from "Order" )
    
    select EmployeeId, LastName, count(*) as AllOrders,sum(lateorder) as LATEORDERS from all_orders a
    inner join Employee b on
    a.EmployeeId = b.Id
    
    group by 1,2
    order by 3
    
    
    
    ;'''
pd.read_sql( Q , con)

,EmployeeId,LastName,AllOrders,LATEORDERS
0,2,Fuller,1805,432
1,9,Dodsworth,1835,451
2,7,King,1839,438
3,8,Callahan,1842,421
4,6,Suyama,1849,437
5,5,Buchanan,1859,432
6,4,Peacock,1907,477
7,1,Davolio,1918,446
8,3,Leverling,1964,419


In [16]:
Q = 'select * from Employee'
pd.read_sql( Q , con)

,Id,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,ReportsTo,PhotoPath
0,1,Davolio,Nancy,Sales Representative,Ms.,1980-12-08,2024-05-01,507 - 20th Ave. E. Apt. 2A,Seattle,North America,98122,USA,(206) 555-9857,5467,None,Education includes a BA in psychology from Col...,2.0,http://accweb/emmployees/davolio.bmp
1,2,Fuller,Andrew,"Vice President, Sales",Dr.,1984-02-19,2024-08-14,908 W. Capital Way,Tacoma,North America,98401,USA,(206) 555-9482,3457,None,Andrew received his BTS commercial in 1974 and...,NaN,http://accweb/emmployees/fuller.bmp
2,3,Leverling,Janet,Sales Representative,Ms.,1995-08-30,2024-04-01,722 Moss Bay Blvd.,Kirkland,North America,98033,USA,(206) 555-3412,3355,None,Janet has a BS degree in chemistry from Boston...,2.0,http://accweb/emmployees/leverling.bmp
3,4,Peacock,Margaret,Sales Representative,Mrs.,1969-09-19,2025-05-03,4110 Old Redmond Rd.,Redmond,North America,98052,USA,(206) 555-8122,5176,None,Margaret holds a BA in English literature from...,2.0,http://accweb/emmployees/peacock.bmp
4,5,Buchanan,Steven,Sales Manager,Mr.,1987-03-04,2025-10-17,14 Garrett Hill,London,British Isles,SW1 8JR,UK,(71) 555-4848,3453,None,Steven Buchanan graduated from St. Andrews Uni...,2.0,http://accweb/emmployees/buchanan.bmp
5,6,Suyama,Michael,Sales Representative,Mr.,1995-07-02,2025-10-17,Coventry House Miner Rd.,London,British Isles,EC2 7JR,UK,(71) 555-7773,428,None,Michael is a graduate of Sussex University (MA...,5.0,http://accweb/emmployees/davolio.bmp
6,7,King,Robert,Sales Representative,Mr.,1992-05-29,2026-01-02,Edgeham Hollow Winchester Way,London,British Isles,RG1 9SP,UK,(71) 555-5598,465,None,Robert King served in the Peace Corps and trav...,5.0,http://accweb/emmployees/davolio.bmp
7,8,Callahan,Laura,Inside Sales Coordinator,Ms.,1990-01-09,2026-03-05,4726 - 11th Ave. N.E.,Seattle,North America,98105,USA,(206) 555-1189,2344,None,Laura received a BA in psychology from the Uni...,2.0,http://accweb/emmployees/davolio.bmp
8,9,Dodsworth,Anne,Sales Representative,Ms.,1998-01-27,2026-11-15,7 Houndstooth Rd.,London,British Isles,WG2 7LT,UK,(71) 555-4444,452,None,Anne has a BA degree in English from St. Lawre...,5.0,http://accweb/emmployees/davolio.bmp
